<a href="https://colab.research.google.com/github/romulo-souza/IA/blob/main/Aprendizado_De_Maquina_Supervisionado/Ajustes_de_Hiperparametros/GridSearch_DecisionTree_BreastCancer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Base de Dados: Breast Cancer
* Base de dados: https://www.kaggle.com/datasets/uciml/breast-cancer-wisconsin-data (baixar e colocar em arquivos no Drive)
* Classe: Diagnosis (M = malignant, B = benign)
* Usaremos o algoritmo de arvore de decisão (decision tree) do scikitlearn. LINK: https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html

In [12]:
import numpy as np
import pandas as pd

# Modelo machine learning
from sklearn.model_selection import train_test_split #amostragem por holdout
from sklearn.preprocessing import StandardScaler #normalização, nesse caso será z score
from sklearn.tree import DecisionTreeClassifier #Algortimo de classificação é a arvore de decisão
from sklearn.preprocessing import LabelEncoder #transformar dados categoricos (classe diagnosis) em numéricos

# Amostragem por Validação Cruzada
from sklearn.model_selection import (
    KFold, #modelo de validação cruzada
    LeaveOneOut, #modelo de validação cruzada
    StratifiedKFold, #modelo de validação cruzada
    cross_validate #classe, função que realiza a validação cruzada
)

# Métricas
from sklearn.metrics import (recall_score,
                             accuracy_score,
                             precision_score,
                             f1_score)
from sklearn.metrics import classification_report # Extrato geral das matricas de classificação

In [13]:
#Função para carregar/gerar Base de Dados (Dataframe)
def carregaBaseDados(nome):
  return pd.read_csv(nome)#retorna um dataframe


## Pré-processamento
* Parametros da função
* dataframe - > dataframe que foi retornado na função carregaBaseDados
* rem_cols - > Colunas a serem removidas, serão passadas em forma de uma lista
* class_column -> Nome da coluna que é a classe alvo, nesse caso é a coluna diagnosis
* normalization_cols -> Especificar quais colunas serão normalizadas,  serão passadas em forma de uma lista

obs.: A normalização deve ser aplicada apenas às colunas que são originalmente numéricas na base de dados. As colunas categóricas, mesmo após serem transformadas em números inteiros pelo LabelEncoder, não devem ser normalizadas. Isso ocorre porque o LabelEncoder atribui números inteiros arbitrários às categorias, mas esses números não têm uma ordem ou escala significativa. Normalizar esses dados pode introduzir distorções que não fazem sentido para o modelo.



In [14]:
def preProcessamento(dataframe, rem_cols, class_column, normalization_cols):

  #Remoção das colunas irrelevantes
  dataframe.drop(rem_cols, axis = 1, inplace = True) #axis = 1 pois é uma coluna, inplace = True para substituir o dataframe original por esse

  #Transformar dados categóricos da classe 'diagnosis' em dados numéricos
  le = LabelEncoder()
  dataframe[class_column] = le.fit_transform(dataframe[class_column]) #os valores categoricos da coluna que tem a classe 'diagnosis' serão substiuídos por uma coluna com valores discretizados (numericos)

  #Normalização dos dados
  scaler = StandardScaler()
  dataframe[normalization_cols] = scaler.fit_transform(dataframe[normalization_cols]) #Faz a normalização das colunas selecionadas e já coloca no dataframe

  return dataframe #retorna o dataframe após as modificaçoes do pre-processamento

In [15]:
df = carregaBaseDados("data.csv")


In [17]:
df.info() #verificar a nossa base de dados para o pre-processamento, percebe-se que todas colunas estao com 569 amostras com exceção da coluna 32 que nao possui nada(gerado por ruído/lixo), ou seja precisará ser removida. Outra coluna que pode ser removida é o id

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 569 entries, 0 to 568
Data columns (total 33 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   id                       569 non-null    int64  
 1   diagnosis                569 non-null    object 
 2   radius_mean              569 non-null    float64
 3   texture_mean             569 non-null    float64
 4   perimeter_mean           569 non-null    float64
 5   area_mean                569 non-null    float64
 6   smoothness_mean          569 non-null    float64
 7   compactness_mean         569 non-null    float64
 8   concavity_mean           569 non-null    float64
 9   concave points_mean      569 non-null    float64
 10  symmetry_mean            569 non-null    float64
 11  fractal_dimension_mean   569 non-null    float64
 12  radius_se                569 non-null    float64
 13  texture_se               569 non-null    float64
 14  perimeter_se             5

In [18]:
df.columns

Index(['id', 'diagnosis', 'radius_mean', 'texture_mean', 'perimeter_mean',
       'area_mean', 'smoothness_mean', 'compactness_mean', 'concavity_mean',
       'concave points_mean', 'symmetry_mean', 'fractal_dimension_mean',
       'radius_se', 'texture_se', 'perimeter_se', 'area_se', 'smoothness_se',
       'compactness_se', 'concavity_se', 'concave points_se', 'symmetry_se',
       'fractal_dimension_se', 'radius_worst', 'texture_worst',
       'perimeter_worst', 'area_worst', 'smoothness_worst',
       'compactness_worst', 'concavity_worst', 'concave points_worst',
       'symmetry_worst', 'fractal_dimension_worst', 'Unnamed: 32'],
      dtype='object')

In [19]:
df = preProcessamento(df, ['id','Unnamed: 32'], 'diagnosis', ['radius_mean', 'texture_mean', 'perimeter_mean',
       'area_mean', 'smoothness_mean', 'compactness_mean', 'concavity_mean',
       'concave points_mean', 'symmetry_mean', 'fractal_dimension_mean',
       'radius_se', 'texture_se', 'perimeter_se', 'area_se', 'smoothness_se',
       'compactness_se', 'concavity_se', 'concave points_se', 'symmetry_se',
       'fractal_dimension_se', 'radius_worst', 'texture_worst',
       'perimeter_worst', 'area_worst', 'smoothness_worst',
       'compactness_worst', 'concavity_worst', 'concave points_worst',
       'symmetry_worst', 'fractal_dimension_worst'])

In [24]:
df.info() #verificar como ficou após o pre-processamento

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 569 entries, 0 to 568
Data columns (total 31 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   diagnosis                569 non-null    int64  
 1   radius_mean              569 non-null    float64
 2   texture_mean             569 non-null    float64
 3   perimeter_mean           569 non-null    float64
 4   area_mean                569 non-null    float64
 5   smoothness_mean          569 non-null    float64
 6   compactness_mean         569 non-null    float64
 7   concavity_mean           569 non-null    float64
 8   concave points_mean      569 non-null    float64
 9   symmetry_mean            569 non-null    float64
 10  fractal_dimension_mean   569 non-null    float64
 11  radius_se                569 non-null    float64
 12  texture_se               569 non-null    float64
 13  perimeter_se             569 non-null    float64
 14  area_se                  5

In [25]:
#Separar os atributos (previsores) da classe (diagnosis) (X -> previsores, y -> classe)
def separaClasse(dataframe, classe):
  X = dataframe.drop(classe, axis = 1)
  y = dataframe[classe]
  return X,y

## Amostragem Holdout

In [26]:
#Separa os conjuntos em treino e teste (70%/30%)
def separaTreinoTeste(X, y):
  return train_test_split(X,y,test_size=0.3)

##Amostragem por validação cruzada, no caso 'K-fold Cross-validation'

Parametros do cross_validate():
* O model a ser passado é uma string, porem para ele interpretar como uma função de modelo de algoritmo que será executado precisamos usar o 'eval', que quebra essa string e a transforma em uma função a ser
* X,y sao os previsores e a classe respectivamente
* Scoring é a métrica que será utilizada para verificar o desempenho do classificador
* cv é o tipo de validação cruzada que estamos usando

In [27]:
def KFCross(model, X, y):
  kf = KFold(n_splits = 10, shuffle = True) #n_splits -> numeros de partições (folds). shuffle -> para cada partição faz um novo embaralhamento para garantir que os dados estejam independentes entre si
  #objeto que fará a validação cruzada
  clf = cross_validate(
      eval(model),
      X,y,
      scoring = 'accuracy',
      cv = kf
  )
  return clf


##Amostragem por validação cruzada, no caso 'Stratified K-fold Cross-validation'

In [28]:
def Skf(model, X, y):
  skf = StratifiedKFold(n_splits = 10, shuffle = True)
  #objeto que fará a validação cruzada
  clf = cross_validate(
      eval(model),
      X,y,
      scoring = 'accuracy',
      cv = skf
  )
  return clf

##Modelo preditivo

In [29]:
#Gera o modelo preditivo (será usado no metodo holdout)
def geraModelo(modelo, X_train, y_train):
  modelo = eval(modelo) #modelo do parametro será passado por uma string, entao precisamos fazer seu eval para ter efeito no codigo
  modelo.fit(X_train, y_train) #treina o modelo
  return modelo


##Métricas

In [30]:
def metricaReport(y_test, y_pred): #y_test ->gabarito do conjunto teste, y_pred -> o que foi predito pelo classificador(algoritmo)
  print(classification_report(y_test, y_pred))#imprimir no formato da função classification report

## Testando com método Holdout

In [31]:
#Separar atributos previsores e classe
X, y = separaClasse(df, 'diagnosis')

#Gerar conjunto treino e teste
X_train, X_test, y_train, y_test = separaTreinoTeste(X, y) #método Holdout

In [32]:
#Avalia os dados (nesse caso o modelo é de arvore de decisão)
#Aqui estamos utilizando o método de amostragem Holdout
modelo = geraModelo('DecisionTreeClassifier()', X_train, y_train)
score = modelo.score(X_test, y_test)#retorna automaticamente a acurácia do modelo (ja faz o predict internamente)
print(score)

0.9473684210526315


In [33]:
#Avalia o modelo com mais métricas e detalhes
y_pred = modelo.predict(X_test) #retorna as saídas preditas pelo classificador
metricaReport(y_test, y_pred)

              precision    recall  f1-score   support

           0       0.99      0.93      0.96       111
           1       0.88      0.98      0.93        60

    accuracy                           0.95       171
   macro avg       0.94      0.96      0.94       171
weighted avg       0.95      0.95      0.95       171



##Testando com métodos de validação cruzada
* A função 'cross_validate', dentro das funções criadas 'KFCross' e 'Skf', retorna um dicionário, e dentro desse dicionário estamos mostrando somente os scores do conjunto teste(chave 'test_score') e mostra também a média dos scores dos testes, pois são 10 testes nesse caso.
* De forma geral, é mais comum se apoiar por abordagens de validação cruzada do que pelo método Holdout para se fazer as análises

In [34]:
#KF
cv = KFCross('DecisionTreeClassifier()', X,y)
print(f"{cv['test_score']}\nMedia: {np.mean(cv['test_score'])}")

[0.9122807  0.96491228 0.9122807  0.94736842 0.92982456 0.89473684
 0.98245614 0.87719298 0.98245614 0.91071429]
Media: 0.931422305764411


In [35]:
#KF estratificado
cv = Skf('DecisionTreeClassifier()', X,y)
print(f"{cv['test_score']}\nMedia: {np.mean(cv['test_score'])}")

[0.94736842 0.92982456 0.94736842 0.96491228 0.92982456 0.94736842
 0.85964912 0.92982456 0.96491228 0.92857143]
Media: 0.9349624060150378


## Testando com técnicas de ajustes de hiperparâmetros -> Podem promover um ganho significativo no score (acuracidade) do modelo quando comparado ao classificador sendo utilizado de maneira padrão/"default"
- Abordagens disponíveis no scikit-learn:
    - GridSearchCV: considera exaustivamente todas as combinações de parâmetros (CV - faz aplicação do método de validação cruzada), pode ter resultados melhores do que o randomizedSearch, pois considera todas combinações possíveis. Demora mais tambem;
    - RandomizedSearchCV: pesquisa aleatória de parâmetros, em que cada configuração é amostrada a partir de uma distribuição de possíveis valores de parâmetro. (CV - faz aplicação do método de validação cruzada)

In [37]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
import sys

#GridSearchCV
- https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html
- https://vitalflux.com/grid-search-explained-python-sklearn-examples/

Utilizaremos o classificador Arvore de decisão, para o GridSearchCV é necessário especificar qual é o classificador usado (parametro estimator) e tambem o parametro param_grid que é um dicionário com os nomes dos parametros e de seus possiveis valores a serem utilizados que o classificador em questão possui.

Obs.: Escolhemos alguns parametros do DecisionTreeClassifier

Decision Tree: https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html

In [38]:
DT = DecisionTreeClassifier() #Especificar qual classificador estamos usando
param_grid = {'criterion': ['gini', 'entropy', 'log_loss'],
              'splitter': ['best', 'random'],
              'max_features': [sys.maxsize, 1.0, 'sqrt', 'log2', None]
              } #especificar um dicionário com os nomes dos parametros que utilizaremos do DecisionTreeClassifier bem como seus possiveis valores em formato de lista (esturutra do dicionario-> {chave: valor})

* max_features -> O número maximo de features (características) a serem consideradas ao procurar a melhor divisão.
se for inteiro, obtemos o maior inteiro pela classe sys usando 'sys.maxsize', se for float o maximo de um float é '1.0', tambem pode ser usado raiz quadrada 'sqrt' e 'log2' ou entao 'None' (não é passado nenhum valor para esse parametro) o qual considera que o numero maximo de features é igual a quantidade total de características.

In [39]:
#instanciar objeto do GridSearchCV
g_search = GridSearchCV(estimator = DT, param_grid = param_grid,
                        cv = 10, refit = True) #cv = 10 -> quantidade de folds (partições) da validação cruzada, Refit = True -> Ao término da busca do GridSearchCV, que usa o método de validação cruzada, ele pega o melhor resultado (melhor combinação possível) e retreina todo o modelo para a base inteira, aplicando o metodo holdout nesse caso



## Treinar o GridSearchCV com X_train e y_train feito pelo train_test_split inicalmente no código (abordagem Holdout)

In [40]:
g_search.fit(X_train, y_train) #treino holdout
print(g_search.best_params_) #melhor combinção

{'criterion': 'log_loss', 'max_features': 'sqrt', 'splitter': 'best'}


In [41]:
#mostrar os atributos que o g_search gera, ele retorna um dicionário
g_search.cv_results_.keys()



dict_keys(['mean_fit_time', 'std_fit_time', 'mean_score_time', 'std_score_time', 'param_criterion', 'param_max_features', 'param_splitter', 'params', 'split0_test_score', 'split1_test_score', 'split2_test_score', 'split3_test_score', 'split4_test_score', 'split5_test_score', 'split6_test_score', 'split7_test_score', 'split8_test_score', 'split9_test_score', 'mean_test_score', 'std_test_score', 'rank_test_score'])

In [42]:
g_search.cv_results_['mean_test_score'] #média dos testes das partições (splits)

array([0.92724359, 0.93730769, 0.93230769, 0.93737179, 0.95      ,
       0.93237179, 0.9224359 , 0.90967949, 0.93717949, 0.9499359 ,
       0.93974359, 0.94974359, 0.93974359, 0.92737179, 0.94224359,
       0.91467949, 0.93724359, 0.91480769, 0.94224359, 0.93724359,
       0.93974359, 0.94230769, 0.94224359, 0.94230769, 0.96487179,
       0.91961538, 0.92987179, 0.92705128, 0.93724359, 0.91224359])

são 30 valores, pois faz todas combinações possíveis usando os valores  dos parametros que foram passado em 'param_grid', entao em 'criterion' temos 3 possiveis valores, em 'splitter temos 2 possiveis valores e em 'max_features' temos 5 possíveis valores, logo todas combinações possíveis será = 3x2x5 = 30

Para cada combinação rodará a validação cruzada com 10 partições, entao cada resultado apresentado é a média de cada 10 partições para cada combinação


In [43]:
#melhor média de score para o conjunto treino (melhor acuracidade)
print(g_search.best_score_)
#índice onde se encontra essa melhor média de score do conjunto treino
print(g_search.best_index_)
#melhor combinção usada
print(g_search.best_params_)

0.9648717948717949
24
{'criterion': 'log_loss', 'max_features': 'sqrt', 'splitter': 'best'}


In [44]:
#Verificar a acuracidade (score) do algoritmo para o conjunto teste, o qual o alg. ainda não conhece
model = g_search.best_estimator_ #pega o melhor modelo dentre as combinações feitas (Arvore de decisão treinada com os melhores parametros encontrados)
model.score(X_test, y_test) #aplicar o modelo no conjunto teste (X_test e y_test), devolve a acuracidade


0.9473684210526315

## Treinar o GridSearchCV com validação cruzada, que já faz as divisões de conjunto treino e teste automaticamente. Usa-se a base inteira (X, y)

In [45]:
g_search.fit(X, y) #treinar para a base inteira
print(g_search.best_params_) #melhor combinção

{'criterion': 'log_loss', 'max_features': 9223372036854775807, 'splitter': 'random'}


In [46]:
# melhor media de score com validação cruzada
print(g_search.best_score_)
# indice onde se encontra essa melhor media de score
print(g_search.best_index_)

0.950814536340852
21
